In [8]:
import os
import re
#import json
from openai import OpenAI
from glob import glob
from pymilvus import model as milvus_model
from pymilvus import MilvusClient
from tqdm import tqdm


# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

text_lines = []

for file_path in glob("./mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    # parts = re.split(r"# |* ", file_text)
    text_lines += file_text.split("* ")






In [10]:
print(len(file_text))
print(len(text_lines))

29239
388


In [14]:
milvus_client = MilvusClient(uri="./milvus_mfd.db")
collection_name = "mfd_rag_collection"
embedding_dim = 768
embedding_model = milvus_model.DefaultEmbeddingFunction()

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

data = []
doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

question = "依据我国明法，个人宅基地和自留地，个人享有有哪些相关权益?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

#retrieved_lines_with_distances = [
#    (res["entity"]["text"], res["distance"]) for res in search_res[0]
#]
#print(json.dumps(retrieved_lines_with_distances, indent=4))

#context = "\n".join(
#    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
#)

SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请回答用 <question> 标签括起来的问题。
<question>
{question}
</question>
"""

response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

Creating embeddings: 100%|██████████| 388/388 [00:00<00:00, 814102.03it/s]


根据我国《民法典》等相关法律规定，个人对宅基地和自留地享有以下主要权益：

1. **宅基地使用权**  
   - 依法享有占有、使用宅基地的权利，可以在宅基地上建造住宅及其附属设施。  
   - 宅基地使用权可以依法由符合条件的继承人继承。  

2. **自留地经营权**  
   - 对自留地享有长期经营使用权，可自主安排种植、养殖等农业生产。  
   - 自留地上的收益归农户所有。  

3. **其他权益**  
   - 宅基地和自留地不得被非法收回或侵占。  
   - 因公共利益需要征收时，有权依法获得合理补偿。  

需注意，宅基地和自留地所有权属于农民集体，个人享有的是使用权或经营权，且相关权益的行使需符合土地管理法等法律法规的规定。


In [16]:
question = "依据我国明法，个人的不动产，个人享有有哪些相关权益?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请回答用 <question> 标签括起来的问题。
<question>
{question}
</question>
"""

response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据中国《民法典》的相关规定，个人对不动产享有以下主要权益：

1. **所有权**：个人对其不动产享有占有、使用、收益和处分的权利。

2. **用益物权**：包括：
   - **土地承包经营权**：对承包的土地享有占有、使用和收益的权利。
   - **建设用地使用权**：依法对国家所有的土地享有占有、使用和收益的权利，可以在该土地上建造建筑物、构筑物及其附属设施。
   - **宅基地使用权**：依法对集体所有的土地享有占有和使用的权利，有权依法利用该土地建造住宅及其附属设施。
   - **居住权**：按照合同约定，对他人的住宅享有占有、使用的权利，以满足生活居住的需要。
   - **地役权**：按照合同约定，利用他人的不动产，以提高自己的不动产的效益。

3. **担保物权**：个人可以以其不动产设立抵押权，作为债务的担保。

这些权益的具体内容和行使方式，需遵循《民法典》及其他相关法律法规的规定。
